In [5]:
!pip install pyspark >> None

Условие: есть набор данных о продажах продуктов с информацией о дате продаж, категории продукта, количестве и выручке от продаж.

Используя Apache Spark, загрузите предоставленный набор данных в DataFrame (пример данных ниже).

("2023-11-20", "Electronics", 100, 12000),
("2023-11-21", "Electronics", 110, 13000),
("2023-11-22", "Electronics", 105, 12500),
("2023-11-20", "Clothing", 300, 15000),
("2023-11-21", "Clothing", 280, 14000),
("2023-11-22", "Clothing", 320, 16000),
("2023-11-20", "Books", 150, 9000),
("2023-11-21", "Books", 200, 12000),
("2023-11-22", "Books", 180, 10000)

Столбцы: "date", "category", "quantity", "revenue".

In [6]:
import pyspark

In [7]:
from pyspark.sql import SparkSession

In [8]:
spark = SparkSession.builder.appName("HW3").getOrCreate()

In [47]:
data = [("2023-11-20", "Electronics", 100, 12000),
        ("2023-11-21", "Electronics", 110, 13000),
        ("2023-11-22", "Electronics", 105, 12500),
        ("2023-11-20", "Clothing", 300, 15000),
        ("2023-11-21", "Clothing", 280, 14000),
        ("2023-11-22", "Clothing", 320, 16000),
        ("2023-11-20", "Books", 150, 9000),
        ("2023-11-21", "Books", 200, 12000),
        ("2023-11-22", "Books", 180, 10000)]
df = spark.createDataFrame(data, ["date", "category", "quantity", "revenue"])
df.show()

+----------+-----------+--------+-------+
|      date|   category|quantity|revenue|
+----------+-----------+--------+-------+
|2023-11-20|Electronics|     100|  12000|
|2023-11-21|Electronics|     110|  13000|
|2023-11-22|Electronics|     105|  12500|
|2023-11-20|   Clothing|     300|  15000|
|2023-11-21|   Clothing|     280|  14000|
|2023-11-22|   Clothing|     320|  16000|
|2023-11-20|      Books|     150|   9000|
|2023-11-21|      Books|     200|  12000|
|2023-11-22|      Books|     180|  10000|
+----------+-----------+--------+-------+



С использованием оконных функций, рассчитайте среднее выручки от продаж для каждой категории продукта.

In [10]:
from pyspark.sql.window import Window

In [16]:
from pyspark.sql.functions import avg

In [48]:
window = Window.partitionBy("category")

In [49]:
avgs_df = df.withColumn("avgRevenue", avg("revenue").over(window))
avgs_df.show()

+----------+-----------+--------+-------+------------------+
|      date|   category|quantity|revenue|        avgRevenue|
+----------+-----------+--------+-------+------------------+
|2023-11-20|      Books|     150|   9000|10333.333333333334|
|2023-11-21|      Books|     200|  12000|10333.333333333334|
|2023-11-22|      Books|     180|  10000|10333.333333333334|
|2023-11-20|   Clothing|     300|  15000|           15000.0|
|2023-11-21|   Clothing|     280|  14000|           15000.0|
|2023-11-22|   Clothing|     320|  16000|           15000.0|
|2023-11-20|Electronics|     100|  12000|           12500.0|
|2023-11-21|Electronics|     110|  13000|           12500.0|
|2023-11-22|Electronics|     105|  12500|           12500.0|
+----------+-----------+--------+-------+------------------+



Примените операцию pivot для того, чтобы преобразовать полученные данные таким образом, чтобы в качестве строк были категории продуктов, в качестве столбцов были дни, а значениями были средние значения выручки от продаж за соответствующий день

In [23]:
from pyspark.sql.functions import col

In [50]:
res_df = avgs_df.groupby("category").pivot("date").mean("revenue")

In [51]:
res_df.show()

+-----------+----------+----------+----------+
|   category|2023-11-20|2023-11-21|2023-11-22|
+-----------+----------+----------+----------+
|Electronics|   12000.0|   13000.0|   12500.0|
|   Clothing|   15000.0|   14000.0|   16000.0|
|      Books|    9000.0|   12000.0|   10000.0|
+-----------+----------+----------+----------+

